This program will take the player's top 100 best plays and plot the year
the map was ranked and how much raw pp it gave the player

Add your API Key onto the API_KEY variable and osu!id#(s) in user_id variable
run every block


In [1]:
import json
import urllib.parse
import urllib.request
import urllib.error

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime


In [2]:
#Insert your own API key here
API_KEY = ''


In [3]:
def access_website(url):
    #accessing the website
    response = None
    
    try:
        
        response = urllib.request.urlopen(url)
        data = response.read()
        html = data.decode("UTF-8")
        return json.loads(html)

    finally:
        if response != None:
            response.close()
            

In [25]:
def url_maker(item_id, api_type):
    if api_type == "player":
        return "https://osu.ppy.sh/api/get_user_best?u="+str(item_id)+"&limit=100&k="+API_KEY
    
    elif api_type == "beatmap":
        return "https://osu.ppy.sh/api/get_beatmaps?b="+item_id+"&k="+API_KEY
    
    return "https://osu.ppy.sh/api/get_user?u="+str(item_id)+"&type=id&k="+API_KEY
        

In [11]:
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')



#so we can get a mindate on x axis (first ranked map: Disco Prince)
mindate = np.datetime64(access_website(url_maker("75","beatmap"))[0]["approved_date"]) 




In [ ]:
#Insert list of player's ID here
user_id = []
colors = ["red","black","green","purple","yellow","blue","pink"]

plays = dict()

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_xlim(mindate, datetime.now())

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)
plt.xlabel("Map's Date When Ranked")
plt.ylabel("Raw pp From Map")



for k, player in enumerate(user_id):
    url = url_maker(user_id[k], "player")
    json_data = access_website(url)
    
    for play in json_data:
        url = url_maker(play["beatmap_id"],"beatmap")
        json_beatmap = access_website(url)
        date = np.datetime64(json_beatmap[0]["approved_date"], 'M')
        
        plt.plot(date,float(play["pp"]),'ro',c=colors[k])
    #lame way to make a legend only have one
    url = url_maker(user_id[k], "name")
    name = access_website(url)[0]["username"]
    plt.plot(date,float(play["pp"]),'ro',c=colors[k],label=name)

        
plt.legend(prop={'size': 10})

plt.show()